<a href="https://colab.research.google.com/github/kargaranamir/issue-tagger/blob/main/LSTM_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Data Download

! wget https://machinehack-be.s3.amazonaws.com/predict_github_issues_embold_sponsored_hackathon/Embold_Participant%27s_Dataset.zip -O data.zip
! unzip ./data.zip 
! mv ./Embold_Participant\'s_Dataset ./data
! rm -rf ./data/sample\ submission.csv
! rm -rf ./data/embold_test.json

--2022-02-06 21:44:12--  https://machinehack-be.s3.amazonaws.com/predict_github_issues_embold_sponsored_hackathon/Embold_Participant%27s_Dataset.zip
Resolving machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)... 52.219.66.56
Connecting to machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)|52.219.66.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102320961 (98M) [application/octet-stream]
Saving to: ‘data.zip’

data.zip            100%[===================>]  97.58M  11.2MB/s    in 11s     

2022-02-06 21:44:24 (9.01 MB/s) - ‘data.zip’ saved [102320961/102320961]

Archive:  ./data.zip
   creating: Embold_Participant's_Dataset/
  inflating: Embold_Participant's_Dataset/sample submission.csv  
  inflating: __MACOSX/Embold_Participant's_Dataset/._sample submission.csv  
  inflating: Embold_Participant's_Dataset/embold_train_extra.json  
  inflating: __MACOSX/Embold_Participant's_Dataset/._embold_train_extra.json  
  inflating: Embo

In [2]:
# install fasttext
! pip install fasttext

     |████████████████████████████████| 68 kB 3.4 MB/s 
  Using cached pybind11-2.9.1-py2.py3-none-any.whl (211 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3128296 sha256=a421330ee2f092b2eaa0489f4414a590b2e602c748297c6179c5b934ff1734db
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


### Import Libraries

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf 

from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import GlobalMaxPool1D, MaxPooling1D, GlobalMaxPooling1D, Conv1D
from sklearn.metrics import classification_report, confusion_matrix


from tensorflow.keras.callbacks import EarlyStopping
import fasttext

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import re
import numpy as np

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string

from tqdm.notebook import tqdm
tqdm.pandas()


# Word Embedding
from gensim.models import KeyedVectors


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Load Data

In [4]:
# merge data
data_small_df = pd.read_json('./data/embold_train.json').reset_index(drop=True)
data_large_df = pd.read_json('./data/embold_train_extra.json').reset_index(drop=True)
data_df = data_small_df.append(data_large_df)
data_df['text'] = data_df['title']+' '+data_df['body']
data_df['text_length'] = data_df['text'].apply(lambda text_input: len(text_input.split()))

In [29]:
# Add constraint on data: Debug 
data_df= data_df[:10000]

### Preprocess

In [30]:
stopwords_list = stopwords.words('english')

def clean_text(text, lowercase=True, stop_words=True, links=True, numbers=True):
    text = text.replace("\\r", "")
    if lowercase:
        text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    if links:
        text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    if numbers:
       text = re.sub('\w*\d\w*', '', text)
    if stop_words:
        text = " ".join([word for word in text.split() if word not in stopwords_list])
    return text

In [31]:
data_df['text_clean'] = data_df['text'].progress_apply(lambda text: clean_text(text))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [32]:
label_encoder = LabelEncoder()

X = data_df['text_clean'].values
y = label_encoder.fit_transform(data_df['label'])

X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2


## FastText Embedding

### Download Fasstext Model

In [33]:
!wget https://www.dropbox.com/s/6aaucelizfx7xl6/en_vectors_v3.bin

--2022-02-06 21:47:32--  https://www.dropbox.com/s/6aaucelizfx7xl6/en_vectors_v3.bin
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/6aaucelizfx7xl6/en_vectors_v3.bin [following]
--2022-02-06 21:47:32--  https://www.dropbox.com/s/raw/6aaucelizfx7xl6/en_vectors_v3.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb73641466c926079285ba3cf7d.dl.dropboxusercontent.com/cd/0/inline/BfPm5LXLWm8rVNI_vuBTnWcOdqu3A9-_8CzQxbui39kyEVynkFwfC3egtxMzzaxiTQm6WVa6CGOury2LEBGidQyN9Uh2RDq4GzE30qysT-_owdeJ16wgUNFipj5YlPcBOdBCC6KF7Vme4rO_mHBx--mB/file# [following]
--2022-02-06 21:47:32--  https://ucb73641466c926079285ba3cf7d.dl.dropboxusercontent.com/cd/0/inline/BfPm5LXLWm8rVNI_vuBTnWcOdqu3A9-_8CzQxbui39kyEVynkFwfC3egtxMzzaxiTQm6WVa6CGO

In [34]:
EMBEDDING_LEN = 128 

### Load FastText Model

In [35]:
model_skipgram = fasttext.load_model('en_vectors_v3.bin')

In [36]:
# Fit Keras Tokenizer on X_train

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=3000)
tokenizer.fit_on_texts(X_train)

vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size : {}'.format(vocab_size))

Vocabulary Size : 6345


In [37]:
encoded_comments = tokenizer.texts_to_sequences(X_train)

# example of encoded comments
print("Comment : {}".format(X_train[1]))
print("Corresponding Encoding : {}".format(encoded_comments[1]))

Comment : could graphqlrequestentity make setrequestmethod public method team use framework compose query graphql api call saw setrequestmethod method graphqlrequestentity access package graphqltemplate call change request method program use feignhystrix http call means use nodes compose query understanding enough change setrequestmethod public method initial process finish graphqlrequestentity constructor method could expose set requestmethod query mutate builder
Corresponding Encoding : [26, 1397, 33, 1398, 130, 104, 295, 7, 558, 1943, 151, 51, 36, 918, 1398, 104, 1397, 381, 244, 36, 37, 34, 104, 490, 7, 152, 36, 429, 7, 652, 1943, 151, 919, 430, 37, 1398, 130, 104, 1944, 353, 1945, 1397, 920, 104, 26, 1946, 16, 151, 1101]


In [38]:
# padding
SENT_MAX_LEN = max([len(sent) for sent in encoded_comments])
padded_sequence = pad_sequences(encoded_comments, maxlen=SENT_MAX_LEN, padding='post')
print('Padding Shape: {}'.format(padded_sequence.shape))

Padding Shape: (600, 271)


In [39]:
# initial embedding matrix
embedding_matrix = np.zeros((vocab_size, EMBEDDING_LEN))

for word, i in tokenizer.word_index.items():
  embedding_vector = model_skipgram.get_word_vector(word)
  # words that cannot be found will be set to 0
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

print(f"Embedding Matrix Shape is: {embedding_matrix.shape}")

Embedding Matrix Shape is: (6345, 128)


In [40]:
# Same procedure with a Unique Tokenizer on Evaluation data

tokenizer.texts_to_matrix(X_val)
eval_encoded_comments = tokenizer.texts_to_sequences(X_val)
eval_padded_sequence = pad_sequences(eval_encoded_comments, maxlen=SENT_MAX_LEN, padding='post')

In [41]:
# Same procedure with a Unique Tokenizer on Test data

tokenizer.texts_to_matrix(X_test)
test_encoded_comments = tokenizer.texts_to_sequences(X_test)
test_padded_sequence = pad_sequences(test_encoded_comments, maxlen=SENT_MAX_LEN, padding='post')

## LSTM Model Architecture

In [42]:
# LSTM constants
LSTM_UNITS = 32

In [43]:
model_1 = Sequential()
model_1.add(Embedding(vocab_size, EMBEDDING_LEN, input_length=SENT_MAX_LEN, weights=[embedding_matrix], trainable=True))
model_1.add(Bidirectional(LSTM(EMBEDDING_LEN, return_sequences=True, input_shape=(None, 1))))
model_1.add(Dropout(0.2))
model_1.add(Bidirectional(LSTM(LSTM_UNITS)))
model_1.add(Dropout(0.2))
model_1.add(Dense(EMBEDDING_LEN, activation='relu'))
model_1.add(Dropout(0.1))
model_1.add(Dense(3, activation='sigmoid'))
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_1.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 271, 128)          812160    
                                                                 
 bidirectional_2 (Bidirectio  (None, 271, 256)         263168    
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 271, 256)          0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 128)              

### Fit LSTM Model
You can run the cell bellow as much as you want. keep track on validation accuracy and also change the `epochs`. I got my best result in most of the run times at 5th and 10th epochs.

In [44]:
y_train_hot = np.eye(3)[y_train] # One-Hot
y_val_hot =  np.eye(3)[y_val] # One-Hot
y_test_hot = np.eye(3)[y_test] # One-Hot

In [45]:
model_1.fit(
    padded_sequence, 
    y_train_hot, 
    batch_size=32, 
    epochs=5, 
    validation_data=(eval_padded_sequence, y_val_hot)
    )

Epoch 1/5
19/19 [==============================] - 14s 382ms/step - loss: 0.9638 - accuracy: 0.4583 - val_loss: 0.9779 - val_accuracy: 0.4500
Epoch 2/5
19/19 [==============================] - 5s 287ms/step - loss: 0.9177 - accuracy: 0.4700 - val_loss: 0.9434 - val_accuracy: 0.6350
Epoch 3/5
19/19 [==============================] - 5s 284ms/step - loss: 0.8321 - accuracy: 0.6583 - val_loss: 0.8211 - val_accuracy: 0.6400
Epoch 4/5
19/19 [==============================] - 5s 285ms/step - loss: 0.6592 - accuracy: 0.7517 - val_loss: 0.7764 - val_accuracy: 0.7000
Epoch 5/5
19/19 [==============================] - 5s 285ms/step - loss: 0.5248 - accuracy: 0.8183 - val_loss: 0.8376 - val_accuracy: 0.6500


In [46]:
loss_1, acc_1 = model_1.evaluate(test_padded_sequence, y_test_hot, verbose=0)
print(f'Test Accuracy: {acc_1}')

Test Accuracy: 0.6299999952316284


In [47]:
y_pred_1 = model_1.predict(test_padded_sequence).argmax(axis=1)
print(confusion_matrix(y_true=y_test, y_pred=y_pred_1))
print(classification_report(y_true=y_test, y_pred=y_pred_1))

[[80  8  0]
 [47 46  0]
 [12  7  0]]
              precision    recall  f1-score   support

           0       0.58      0.91      0.70        88
           1       0.75      0.49      0.60        93
           2       0.00      0.00      0.00        19

    accuracy                           0.63       200
   macro avg       0.44      0.47      0.43       200
weighted avg       0.60      0.63      0.59       200



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## CNN Model Architecture

In [48]:
## CNN Constants
KERNEL_SIZE = 3
FILTERS = 256

In [49]:
model_2 = Sequential()
model_2.add(Embedding(vocab_size, embedding_matrix.shape[1], weights=[embedding_matrix], trainable=False))
model_2.add(Conv1D(filters=FILTERS, kernel_size=KERNEL_SIZE, activation='relu'))
model_2.add(GlobalMaxPooling1D())
model_2.add(Dense(FILTERS, activation='relu'))
model_2.add(Dense(3, activation='sigmoid'))
model_2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model_2.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 128)         812160    
                                                                 
 conv1d_1 (Conv1D)           (None, None, 256)         98560     
                                                                 
 global_max_pooling1d_1 (Glo  (None, 256)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_6 (Dense)             (None, 256)               65792     
                                                                 
 dense_7 (Dense)             (None, 3)                 771       
                                                                 
Total params: 977,283
Trainable params: 165,123
Non-trainable params: 812,160
__________________________________________

### Fit CNN Model
You can run the cell bellow as much as you want. keep track on validation accuracy and also change the `epochs`. I got my best result in most of the run times at 5th epoch.

In [26]:
model_2.fit(
    padded_sequence, 
    y_train_hot, 
    batch_size=32, 
    epochs=5, 
    validation_data=(eval_padded_sequence, y_val_hot)
    )

Epoch 1/5
19/19 [==============================] - 7s 25ms/step - loss: 0.9478 - accuracy: 0.4817 - val_loss: 0.9490 - val_accuracy: 0.5500
Epoch 2/5
19/19 [==============================] - 0s 15ms/step - loss: 0.7710 - accuracy: 0.7183 - val_loss: 0.9173 - val_accuracy: 0.6050
Epoch 3/5
19/19 [==============================] - 0s 13ms/step - loss: 0.5875 - accuracy: 0.8033 - val_loss: 0.8755 - val_accuracy: 0.6250
Epoch 4/5
19/19 [==============================] - 0s 13ms/step - loss: 0.3856 - accuracy: 0.9000 - val_loss: 0.7849 - val_accuracy: 0.6800
Epoch 5/5
19/19 [==============================] - 0s 15ms/step - loss: 0.1760 - accuracy: 0.9800 - val_loss: 0.8041 - val_accuracy: 0.6850


In [27]:
loss_2, acc_2 = model_2.evaluate(test_padded_sequence, y_test_hot, verbose=0)
print('Test Accuracy: %f' % (acc_2*100))

Test Accuracy: 64.499998


In [28]:
y_pred_2 = model_2.predict(test_padded_sequence).argmax(axis=1)
print(confusion_matrix(y_true=y_test, y_pred=y_pred_2))
print(classification_report(y_true=y_test, y_pred=y_pred_2))

[[71 17  0]
 [35 58  0]
 [ 8 11  0]]
              precision    recall  f1-score   support

           0       0.62      0.81      0.70        88
           1       0.67      0.62      0.65        93
           2       0.00      0.00      0.00        19

    accuracy                           0.65       200
   macro avg       0.43      0.48      0.45       200
weighted avg       0.59      0.65      0.61       200



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
